In [36]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
#pip install pypyodbc

In [37]:
data = pd.read_csv(r'kiva_loans.csv')
data.head(1)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12,12,NaN,female,irregular,2014-01-01


In [38]:
data.dtypes

id                      int64
funded_amount           int64
loan_amount             int64
activity               object
sector                 object
purpose                object
country_code           object
country                object
region                 object
currency               object
partner_id            float64
posted_time            object
disbursed_time         object
funded_time            object
term_in_months          int64
lender_count            int64
tags                   object
borrower_genders       object
repayment_interval     object
entryDate              object
dtype: object

In [6]:
data['disbursed_time'] = pd.to_datetime(data['disbursed_time'])
data['funded_time'] = pd.to_datetime(data['funded_time'])
data['posted_time'] = pd.to_datetime(data['posted_time'])

data.head(1)



,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12,12,NaN,female,irregular,2014-01-01


In [7]:
#data.dtypes

In [8]:
# Extract date and time components into new columns
data['disbursed_date'] = data['disbursed_time'].dt.date
data['disbursed_timeofday'] = data['disbursed_time'].dt.time
data['funded_date'] = data['funded_time'].dt.date
data['funded_time_timeofday'] = data['funded_time'].dt.time
#data['posted_date'] = data['posted_time'].dt.date
#data['posted_time_timeofday'] = data['posted_time'].dt.time
data.head(1)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,...,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate,disbursed_date,disbursed_timeofday,funded_date,funded_time_timeofday
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,12,12,NaN,female,irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,10:06:32


In [13]:
data.head(2)

,id,funded_amount,loan_amount,activity,sector,purpose,country_code,country,region,currency,...,term_in_months,lender_count,tags,borrower_genders,repayment_interval,entryDate,disbursed_date,disbursed_timeofday,funded_date,funded_time_timeofday
0,653051,300,300,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,...,12,12,NaN,female,irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,10:06:32
1,653053,575,575,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,...,11,14,NaN,"female, female",irregular,2014-01-01,2013-12-17,08:00:00,2014-01-02,09:17:23


In [17]:
from sqlalchemy import create_engine


Trying PYpodbc

In [24]:
import pyodbc

#Loop through all drivers we have access to
for driver in pyodbc.drivers():
    print(driver)


SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server


#### Test code to verify connection to SQL server

In [27]:
import pyodbc
import textwrap

# Define the Price Data.
price_data = [
    [ 2.00, 3.00, 1.00, 2.40, 100.00, '1/2/2019'],
    [ 3.00, 3.00, 5.00, 9.40, 300.00, '2/1/2020'],
    [ 4.00, 2.00, 1.00, 2.40, 200.00, '3/1/2021']
]

# define the server and the database
server = 'localhost' 
database = 'KivaLoans'  

# Define the connection string
cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE='+ database +';\
    Trusted_Connection=yes;'
)

# Create the Cursor.
cursor = cnxn.cursor()

# Loop through to insert each row.
for index, row in enumerate(price_data):
    
    # define an insert query with place holders for the values.
    insert_query = textwrap.dedent('''
        INSERT INTO td_price_data (close_price, high, low, open_price, volume, day_value) 
        VALUES (?, ?, ?, ?, ?, ?);
    ''')
    
    # define the values
    values = (row[0], row[1], row[2], row[3], row[4], row[5])
    
    # insert the data into the database
    cursor.execute(insert_query, values)

# commit the inserts.
cnxn.commit()
    
# grab all the rows from the table
cursor.execute('SELECT * FROM td_price_data')
for row in cursor:
    print(row)
    
# close the cursor and connection  
cursor.close()
cnxn.close()

(2.0, 3.0, 1.0, 2.4, 100.0, datetime.date(2019, 1, 2))
(3.0, 3.0, 5.0, 9.4, 300.0, datetime.date(2020, 2, 1))
(4.0, 2.0, 1.0, 2.4, 200.0, datetime.date(2021, 3, 1))


In [32]:
import pyodbc

# Define the connection string
 # define the server and the database
server = 'localhost' 
database = 'KivaLoans'  

# Define the connection string
cnxn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE='+ database +';\
    Trusted_Connection=yes;'
)

# Create the Cursor.
#cursor = cnxn.cursor()
# Establish the connection
try:
    connection = cnxn
    print("Connection successful!")
except pyodbc.Error as ex:
    print("Connection failed: ", ex)


Connection successful!


In [19]:
#pip install bcpandas

In [39]:
 #Create the connection string for SQLAlchemy
from sqlalchemy import create_engine

#Define server and database name
server = 'localhost' 
database = 'KivaLoans' 

#connection string
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to a new SQL table
table_name = 'kiva_loans_main'
try:
    # 'if_exists' parameter can be 'fail', 'replace', or 'append'
    data.to_sql(name='kiva_loans_main', con=engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully in the '{database}' database.")
except Exception as e:
    print(f"An error occurred: {e}")
    
    

Table 'kloans' created successfully in the 'KivaLoans' database.


In [40]:
import pandas as pd
from bcpandas import to_sql
from sqlalchemy import create_engine



#Define server and database name
server = 'localhost' 
database = 'KivaLoans' 

# Create the connection string for SQLAlchemy
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Define the table name
table_name = 'kiva_loans_main'



# Use bcpandas to write the DataFrame to the SQL table
try:
    to_sql(df=data, name=table_name, schema='dbo', con=engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' created successfully in the '{database}' database.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: to_sql() got an unexpected keyword argument 'name'
